# wps_climdex_fd

In [1]:
import os
from birdy import WPSClient
from pkg_resources import resource_filename

# Ensure we are in the working directory with access to the data
# while os.path.basename(os.getcwd()) != "osprey":
#     os.chdir('../')

In [2]:
quail = WPSClient("http://localhost:5005/wps")

In [3]:
quail.climdex_fd?

Signature:
quail.climdex_fd(
    ci_file,
    ci_name=None,
    loglevel='INFO',
    output_obj='fd',
    output_file='output.rda',
)
Docstring:
Computes the annual count of days when daily minimum temperature is below 0 degrees Celsius

Parameters
----------
ci_file : ComplexData:mimetype:`application/x-gzip`
    File that holds climdexInput object (recommended file extension .rda)
ci_name : string
    Name of the climdexInput obejct
output_obj : string
    Name of the output object
output_file : string
    Filename to store the output (recommended file extension .rda)
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
frost_days_file : ComplexData:mimetype:`application/x-gzip`
    A vector containing the number of frost days for each year
File:      ~/Desktop/quail/docs/source/notebooks/</home/sangwonl/.local/lib/python3.6/site-packages/birdy/client/base.py-1>
Type:      method


In [4]:
output = quail.climdex_fd(
    ci_file = "/home/sangwonl/Desktop/quail/tests/data/climdexInput.rda",
    ci_name="ci",
    output_obj="fd",
    output_file="fd.rda",
)